# Neo4j Generative AI Workshop
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/xpilasneo4j/genai-workshop-2024/blob/main/genai-workshop.ipynb)

In this workshop, you will learn how to use Neo4j Knowledge Graphs to make Large Language Models (LLMs) useful for more real-world use cases.

We walk through an example that uses real-world customer and product data from a fashion, style, and beauty retailer. We show how you can use a knowledge graph to ground an LLM, enabling it to build tailored marketing content personalized to each customer based on their interests and shared purchase histories. We use a pattern called Retrieval-Augmented Generation (RAG) to accomplish this.  Specifically, one that leverages not only vector search but also graph pattern matching and graph machine learning to provide more relevant personalized results to customers.

This notebook walks through the end-to-end process, including:
- Building the knowledge graph
- Vector search & text embedding
- Using graph patterns in Cypher to improve semantic search with context
- Further augmenting semantic search with knowledge graph inference & ML
- Building the LLM chain and demo app for generating content

---

## 1/ Graph Building

## Setup

### Some Logics
1. Make a copy of this notebook in Colab by [clicking here](https://colab.research.google.com/github/xpilasneo4j/genai-workshop-2024/blob/main/genai-workshop.ipynb).
2. Run the pip install below to get the necessary dependencies.  this can take a while. Then run the following cell to import relevant libraries


In [ ]:
%pip install sentence_transformers langchain langchain-openai langchain-community openai tiktoken python-dotenv gradio graphdatascience altair neo4j_tools
%pip install "vegafusion[embed]"

In [ ]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from graphdatascience import GraphDataScience
from neo4j_tools import gds_db_load, gds_utils
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import gradio as gr

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

### Setup Credentials and Environment Variables

There are two things you need here.
1. Start a *Blank Sandbox - Graph Data Science* [Neo4j Sandbox](https://sandbox.neo4j.com/). 
<p></p>
<img src="img/sandbox.png"/>

2. Get your URI and password and plug them in below.  Do not change the Neo4j username.
<p></p>
<img src="img/sandbox-detail.png"/>

3. Get your OpenAI API key.  You can use [this one](https://docs.google.com/document/d/19Lqjd0MqRs088KUVnd23ZrVU9G0OAg-53U72VrFwwms/edit) if you do not have one already

To make this easy, you can write the credentials and env variables directly into the below cell.

In [ ]:
from dotenv import load_dotenv
import os

# Neo4j
NEO4J_URI = 'bolt://34.202.229.218:7687' #change this
NEO4J_PASSWORD = 'terminologies-fire-planet' #change this
NEO4J_USERNAME = 'neo4j'
AURA_DS = False

# AI
LLM = 'gpt-4'

# OpenAI - Required when using OpenAI models
os.environ['OPENAI_API_KEY'] = 'sk-...' #change this

In [ ]:
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists('ws.env'):
    load_dotenv('ws.env', override=True)

    # Neo4j
    NEO4J_URI = os.getenv('NEO4J_URI')
    NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
    NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
    AURA_DS = eval(os.getenv('AURA_DS').title())

    # AI
    LLM = os.getenv('LLM')

## Knowledge Graph Building

<img src="img/hm-banner.png" alt="summary" width="2000"/>

We begin by building our knowledge graph. This workshop will leverage the [H&M Personalized Fashion Recommendations Dataset](https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data), a sample of real customer purchase data that includes rich information around products including names, types, descriptions, department sections, etc.

Below is the graph data model we will use:

<img src="img/data-model.png" alt="summary" width="1000"/>


### Connect to Neo4j

We will use the [Graph Data Science Python Client](https://neo4j.com/docs/graph-data-science-client/current/) to connect to Neo4j. This client makes it convenient to display results, as we will see later.  Perhaps more importantly, it allows us to easily run [Graph Data Science](https://neo4j.com/docs/graph-data-science/current/introduction/) algorithms from Python.

This client will only work if your Neo4j instance has Graph Data Science installed.  If not, you can still use the [Neo4j Python Driver](https://neo4j.com/docs/python-manual/current/) or use Langchain’s Neo4j Graph object that we will see later on.

In [ ]:
from graphdatascience import GraphDataScience

# Use Neo4j URI and credentials according to our setup
gds = GraphDataScience(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    aura_ds=AURA_DS)

# Necessary if you enabled Arrow on the db - this is true for AuraDS
gds.set_database("neo4j")

Test your connection by running the below.  It should output your system information.

In [ ]:
gds.debug.sysInfo()

### Get Data, Create Constraints, and Load

Before loading data into Neo4j, it is usually best practice to create Key or Uniqueness constraints for nodes. These [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) act as an index with some validation on unique id properties and thus make `MATCH` statements run significantly faster. Not doing this can result in a VERY slow ingest, so this is a critical step.

We will be using convenience functions for loading nodes and relationships in batch. As the data load runs, you can see the Cypher statements being used.

In [ ]:
%%time

import pandas as pd
from neo4j_tools import gds_db_load

# get source data - it has been pre-formatted. If you would like to re-generate from source on Kaggle, see the data-prep.ipynb notebook
department_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/department.csv')
product_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/product.csv')
article_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/article.csv')
customer_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/customer.csv')
transaction_df = pd.read_csv('https://storage.googleapis.com/neo4j-workshop-data/genai-hm/transaction.csv')

# create constraints - one uniqueness constraint for each node label
gds.run_cypher('CREATE CONSTRAINT unique_department_no IF NOT EXISTS FOR (n:Department) REQUIRE n.departmentNo IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_product_code IF NOT EXISTS FOR (n:Product) REQUIRE n.productCode IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_article_id IF NOT EXISTS FOR (n:Article) REQUIRE n.articleId IS UNIQUE')
gds.run_cypher('CREATE CONSTRAINT unique_customer_id IF NOT EXISTS FOR (n:Customer) REQUIRE n.customerId IS UNIQUE')

# load nodes
gds_db_load.load_nodes(gds, department_df, 'departmentNo', 'Department')
gds_db_load.load_nodes(gds, article_df.drop(columns=['productCode', 'departmentNo']), 'articleId', 'Article')
gds_db_load.load_nodes(gds, product_df, 'productCode', 'Product')
gds_db_load.load_nodes(gds, customer_df, 'customerId', 'Customer')

# load relationships
gds_db_load.load_rels(gds, article_df[['articleId', 'departmentNo']], source_target_labels=('Article', 'Department'),
                      source_node_key='articleId', target_node_key='departmentNo',
                      rel_type='FROM_DEPARTMENT')
gds_db_load.load_rels(gds, article_df[['articleId', 'productCode']], source_target_labels=('Article', 'Product'),
                      source_node_key='articleId',target_node_key='productCode',
                      rel_type='VARIANT_OF')
gds_db_load.load_rels(gds, transaction_df, source_target_labels=('Customer', 'Article'),
                      source_node_key='customerId', target_node_key='articleId', rel_key='txId',
                      rel_type='PURCHASED')

# convert transaction dates
gds.run_cypher('''
MATCH (:Customer)-[r:PURCHASED]->()
SET r.tDat = date(r.tDat)
''')

# create combined text property. This will help simplify later with semantic search and RAG
gds.run_cypher("""
    MATCH(p:Product)
    SET p.text = '##Product\n' +
        'Name: ' + p.prodName + '\n' +
        'Type: ' + p.productTypeName + '\n' +
        'Group: ' + p.productGroupName + '\n' +
        'Garment Type: ' + p.garmentGroupName + '\n' +
        'Description: ' + p.detailDesc
    RETURN count(p) AS propertySetCount
    """)

---

## 2/ Vector Search

In this section, we will build text embeddings out of product descriptions and demonstrate how to leverage the Neo4j vector index for vector search. We will also introduce the use of [LangChain](https://www.langchain.com/).


### Creating Text Embeddings

To start we need to make embeddings for our product nodes.

First, we will instantiate our embedding model. This notebook has just been tested with OpenAI, but these models are pluggable. You could choose embedding models from other providers, including cloud providers like AWS Bedrock and Vertex AI Generative AI.

In [ ]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
embedding_dimension = 1536

Now let's create a dataframe with a text column to embed.  In this case, we will combine multiple text columns, such as product name, type, description, etc.  This provides the embedding model with more context.  Some products are missing a description (a small minority).  For our intents and purposes we will leave them out. In a more in-depth workflow, you would likely want to impute the missing values.

In [ ]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

product_emb_df = product_df[['productCode', 'prodName', 'productTypeName', 'productGroupName', 'garmentGroupName', 'detailDesc']]
product_emb_df = product_emb_df[product_emb_df.detailDesc.notnull()]

def create_doc(row):
    return f'''
##Product
Name: {row.prodName}
Type: {row.productTypeName}
Group: {row.productGroupName}
Garment Type: {row.garmentGroupName}
Description: {row.detailDesc}
'''

product_emb_df['text'] = product_emb_df.apply(create_doc, axis=1)
product_emb_df = product_emb_df.drop(columns=['prodName', 'productTypeName', 'productGroupName', 'garmentGroupName', 'detailDesc'])
product_emb_df

Now let’s embed the text with OpenAI.  We will chunk this into batches for efficiency.

In [ ]:
%%time

count = 0
embeddings = []
for docs in gds_db_load.chunks(product_emb_df.text, n=500):
    count += len(docs)
    print(f'Embedded {count} of {product_emb_df.shape[0]}')
    embeddings.extend(embedding_model.embed_documents(docs))

# Set as column of dataframe to prepare for loading
product_emb_df['textEmbedding'] = embeddings
product_emb_df

#### Create Vector Properties and Index

Now, we will load the embeddings into Neo4j by MATCHing on ProductCode, then calling the `db.create.setNodeVectorProperty` to set the embedding property. This special function is used to set the properties as floats rather than double precision, which requires more space.  This becomes important as these embedding vectors tend to be long, and the size can add up quickly.

After bulk loading, we will create the vector index. The [Neo4j Vector Index](https://neo4j.com/docs/cypher-manual/current/indexes-for-vector-search/) enables efficient Approximate Nearest Neighbor (ANN) search with vectors. It uses the Hierarchical Navigable Small World (HNSW) algorithm.

In [ ]:
# load vector properties
records = product_emb_df[['productCode', 'textEmbedding']].to_dict('records')
print(f'======  loading Product text embeddings ======')
total = len(records)
print(f'staging {total:,} records')
cumulative_count = 0
for recs in gds_db_load.chunks(records, n=100):
    res = gds.run_cypher('''
    UNWIND $recs AS rec
    MATCH(n:Product {productCode: rec.productCode})
    CALL db.create.setNodeVectorProperty(n, "textEmbedding", rec.textEmbedding)
    RETURN count(n) AS propertySetCount
    ''', params={'recs': recs})
    cumulative_count += res.iloc[0, 0]
    print(f'Set {cumulative_count:,} of {total:,} text embeddings')

#create index
gds.run_cypher('''
CREATE VECTOR INDEX product_text_embeddings IF NOT EXISTS FOR (n:Product) ON (n.textEmbedding)
OPTIONS {indexConfig: {
 `vector.dimensions`: toInteger($dim),
 `vector.similarity_function`: 'cosine'
}}''', params={'dim': embedding_dimension})

gds.run_cypher('CALL db.awaitIndex("product_text_embeddings", 300)')

### Vector Search Using Cypher

To do vector search, we need to:
1. Take the search prompt and convert it to an embedding query vector
2. Use similarity search with that new vector to pull semantically similar documents

Below is an example of converting a search prompt into a query vector. We use our same embedding model to do this.

In [ ]:
#search_prompt = 'denim jeans, loose fit, high-waist'
search_prompt = 'Oversized Sweaters'

query_vector = embedding_model.embed_query(search_prompt)
print(f'query vector length: {len(query_vector)}')
print(f'query vector sample: {query_vector[:10]}')

Now we can take that and use it in a Cypher query with the vector index to retrieve semantically similar documents.

In [ ]:
gds.run_cypher('''
CALL db.index.vector.queryNodes("product_text_embeddings", 10, $queryVector)
YIELD node AS product, score
RETURN product.productCode AS productCode,
    product.text AS text,
    score
''', params={'queryVector': query_vector})

### Vector Search Using Langchain

We can also do this vector search with Langchain, a recommended approach going forward.  To do this, we use the Neo4jVector class and call the below method to set up from an existing index in the graph.

In [ ]:
from langchain.vectorstores.neo4j_vector import Neo4jVector

kg_vector_search = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='product_text_embeddings')

Langchain can handle embedding the query vector and retrieving from Neo4j behind the scenes, making our lives easier.  Langchain uses a similar query as above and retrieves the `text` property we set for each Product node.

In [ ]:
res = kg_vector_search.similarity_search(search_prompt, k=10)
res

In [ ]:
# Visualize as a dataframe
pd.DataFrame([{'document': d.page_content} for d in res])

### Try Yourself

Experiment with your own prompts!

In [ ]:
res = kg_vector_search.similarity_search('type your prompt here!', k=10)
pd.DataFrame([{'document': d.page_content} for d in res])

---

## 3/ Semantic Search with Context (Personalizations)
__Using Graph Patterns to Improve Context in Search & Retrieval__

Above, we saw how you can use the vector index to find semantic similar products in user searches.  This is an extremely powerful tool; however, it is not the end-all be-all.  It doesn't consider much of the customer data and isn't very personalized. Furthermore, some search
prompts, like "Oversized Sweater," are very general and can match a large number of products, many of which won't be relevant to the specific user conducting the search.

We have a rich knowledge graph full of customer information; let's see how to leverage it to improve search experience.

### Explore in Sandbox
To understand how to better leverage our graph, let's explore in Neo4j Browser on our sandbox instance.

#### Exploring the Graph
First, let's validate the schema by calling the below
```
CALL db.schema.visualization()
```

We can also use Cypher to sample the graph. Run the below query in Browser and explore the results:
```
MATCH (p:Product)<-[v:VARIANT_OF]-(a:Article)<-[t:PURCHASED]-(c:Customer)
RETURN * LIMIT 150
```

You should get something that looks like the below.  Notice the multi-hop connections between customers based on purchases. This is valuable information encoded in our graph!

<img src="img/sample-query.png" alt="summary" width="1000"/>

#### Understanding Shared Customer Behavior

Now let's consider a single customer's purchase history.  We will choose the below customer by setting customerId as a parameter.

```
:params {customerId:'daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0'}
```

Then we can run the below Cypher to pull history:

```
MATCH(c:Customer {customerId: $customerId})-[t:PURCHASED]->(:Article)
-[:VARIANT_OF]->(p:Product)
RETURN p.productCode AS productCode,
    p.prodName AS prodName,
    p.productTypeName AS productTypeName,
    p.garmentGroupName AS garmentGroupName,
    p.detailDesc AS detailDesc,
    t.tDat AS purchaseDate
ORDER BY t.tDat DESC
```
Expected results:
<img src="img/purchase-history.png" alt="summary" width="1000"/>

These purchases are ordered by transaction date. The most recent purchases should be the "Tove Top" and the "Rosemary Dress".

Now let's consider just the latest products in the above list and see what else we could recommend to customers who liked them.  The following Cypher query provides potential answers by finding the most popular products among customers who purchased these.

```
//pull the latest purchases
MATCH(c:Customer {customerId: $customerId})-[t:PURCHASED]->()
WITH max(t.tDat) AS latestPurchases
//find related products based on customer purchases
MATCH(c:Customer {customerId: $customerId})-[:PURCHASED {tDat: latestPurchases}]->(:Article)<-[:PURCHASED]-(:Customer)-[:PURCHASED]->(:Article)
    -[:VARIANT_OF]->(p:Product)
RETURN p.productCode AS productCode,
    p.prodName AS prodName,
    p.productTypeName AS productTypeName,
    p.garmentGroupName AS garmentGroupName,
    count(*) AS commonPurchaseScore,
    p.detailDesc AS detailDesc
ORDER BY commonPurchaseScore DESC
```

Expected results:
<img src="img/related-products.png" alt="summary" width="1000"/>

__You will see that some of the above results seem intuitive...but not all of them right away...and that is exactly the point!
There is information encoded inside the knowledge graph about customer preferences that isn't inferable from the product text documents.__

__This is one example of where enterprise-specific data, expressed as structured relationships, contains critical information that is impossible to find elsewhere. This is why, for many real-world applications, you should consider backing semantic search and GenAI with Knowledge Graphs.__

Now let’s see how to apply this pattern in our semantic search and retrieval!

### Personalizing Results Based on Customer Behavior in the Graph

As we saw in Browser, an important piece of information expressed in this graph, but not directly in the product documents and text embeddings, is customer purchasing behavior.  We saw that we can use graph patterns in Cypher to extract insights from these. Now that we know how this pattern works, we can apply it to our semantic search to make results more personalized.

To do this, we append a MATCH statement to the end of our initial vector search query.  Basically, once the product documents are returned, we can re-calculate how they would score according to the query above and use that to re-rank the search results.

Langchain makes this easy by allowing for a `retrieval_query` argument where we can put in the pattern we need.

In [ ]:
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"

kg_personalized_search = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='product_text_embeddings',
    retrieval_query=f"""
    WITH node AS product, score AS searchScore

    OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
    -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {{customerId: '{CUSTOMER_ID}'}})

    WITH count(a) AS purchaseScore, product.text AS text, searchScore, product.productCode AS productCode
    RETURN text,
        (1+purchaseScore)*searchScore AS score,
        {{productCode: productCode, purchaseScore:purchaseScore, searchScore:searchScore}} AS metadata
    ORDER BY purchaseScore DESC, searchScore DESC LIMIT 15
    """)

Now let's run it to see if/how our results have changed.

In [ ]:
res = kg_personalized_search.similarity_search(search_prompt, k=100)

# Visualize as a dataframe
pd.DataFrame([{'productCode': d.metadata['productCode'],
               'document': d.page_content,
               'searchScore': d.metadata['searchScore'],
               'purchaseScore': d.metadata['purchaseScore']} for d in res])

---

## 4/ Graph Data Science & ML (Recommendations)

We saw above how to use graph pattern matching to personalize semantic search and make it more contextually relevant.

In addition to this, we also have [Graph Data Science algorithms and machine learning](https://neo4j.com/docs/graph-data-science/current/introduction/) which allows you to enrich your knowledge graph with additional properties, relationships, and graph metrics. These can in-turn be leveraged in search and retrieval to improve and augment results.

We will walk through an example of this below, where we use Graph Data Science to augment retrieval with additional product recommendations.


Below, we use graph machine learning to create relationships that can help us make personalized recommendations based on purchase history.


We do this by leveraging Node Embeddings with K-Nearest Neighbor (KNN). Specifically we:
1. Use Node embeddings to encode the similarity between articles based on shared customer search history
2. Take those node embeddings as input to KNN, an unsupervised learning technique, to link the most similar products together with a "CUSTOMERS_ALSO_LIKE" relationship.
3. We can then use Cypher patterns at query time to grab recommended items based on a customer's recent purchase history.
This process helps scale memory-based recommendation techniques to larger datasets.

In [ ]:
%%time
from neo4j_tools import gds_utils

#clear past GDS analysis in the case of re-running
gds_utils.clear_all_gds_graphs(gds)
gds_utils.delete_relationships('CUSTOMERS_ALSO_LIKE', gds, src_node_label='Article')


# graph projection - project co-purchase graph into analytics workspace
gds.run_cypher('''
   MATCH (a1:Article)<-[:PURCHASED]-(:Customer)-[:PURCHASED]->(a2:Article)
   WITH gds.graph.project("proj", a1, a2,
       {sourceNodeLabels: labels(a1),
       targetNodeLabels: labels(a2),
       relationshipType: "COPURCHASE"}) AS g
   RETURN g.graphName
   ''')
g = gds.graph.get("proj")

# create FastRP node embeddings
gds.fastRP.mutate(g, mutateProperty='embedding', embeddingDimension=128, randomSeed=7474, concurrency=4, iterationWeights=[0.0, 1.0, 1.0])

# draw KNN
knn_stats = gds.knn.write(g, nodeProperties=['embedding'], nodeLabels=['Article'],
                  writeRelationshipType='CUSTOMERS_ALSO_LIKE', writeProperty='score',
                  sampleRate=1.0, initialSampler='randomWalk', concurrency=1, similarityCutoff=0.75, randomSeed=7474)

# write embeddings back to database to introspect later
gds.graph.writeNodeProperties(g, ['embedding'], ['Article'])

# clear graph projection once done
g.drop()

# output knn stats
knn_stats

### Visualize Node Embeddings
To better help understand what the Node embeddings are doing, let’s pull some back and visualize them!

__NOTE__: The visualization below should be pre-rendered to cut down on runtime. Running the TSNE cell can take ~5 minutes.

In [ ]:
graph_emb_df = gds.run_cypher('''
MATCH (p:Product)<-[:VARIANT_OF]-(a:Article)-[:FROM_DEPARTMENT]-(d)
RETURN a.articleId AS articleId,
    p.prodName AS productName,
    p.productTypeName AS productTypeName,
    d.departmentName AS departmentName,
    d.sectionName AS sectionName,
    p.detailDesc AS detailDesc,
    a.embedding AS embedding
''')

#view a sample
graph_emb_df.loc[:3, ['articleId', 'embedding']]

In [ ]:
# Skip this for Colab Workshop
# import numpy as np
# from sklearn.manifold import TSNE
#
# df = graph_emb_df.copy()
# filtered_node_df = df[df.embedding.apply(lambda x: np.count_nonzero(x) > 0)].reset_index(drop=True)
# # instantiate the TSNE model
# tsne = TSNE(n_components=2, random_state=7474, init='random', learning_rate="auto")
# # Use the TSNE model to fit and output a 2-d representation
# E = tsne.fit_transform(np.stack(filtered_node_df['embedding'], axis=0))
#
# coord_df = pd.concat([filtered_node_df, pd.DataFrame(E, columns=['x', 'y'])], axis=1)
# coord_df

In [ ]:
# Skip this for Colab Workshop
# import altair as alt
# from sklearn.manifold import TSNE
#
# alt.data_transformers.disable_max_rows()
# chart = alt.Chart(coord_df.sample(n=5000, random_state=7474)).mark_circle(size=60).encode(
#     x='x',
#     y='y',
#     tooltip=['productName', 'productTypeName', 'departmentName' , 'sectionName', 'detailDesc']
# ).properties(title="Article Embedding (2D Representation)", width=750, height=700)
#
# chart = chart.configure_axis(titleFontSize=20)
# chart.configure_legend(labelFontSize = 20)
# chart

### Personalized Recommendations

Now, let's construct a KG store to retrieve recommendations for a user.  This need not be based on a user prompt or vector search. Instead, we will make it purely based on purchase history.

In [ ]:
from langchain.graphs import Neo4jGraph

kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

res = kg.query('''
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.productCode AS productCode,
        product.prodName AS prodName,
        product.productTypeName AS productType,
        product.text AS document,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    ''', params={'customerId': CUSTOMER_ID, 'k':15})

#visualize as dataframe. result is list of dict
pd.DataFrame(res)

---

## 5/ LLM Powered Content Generator

Let's use an LLM to automatically generate content for targeted marketing campaigns grounded with our knowledge graph using the above tools.
Here is a quick example for generating promotional messages, but you can create all sorts of content with this!

For our first message, let's consider a scenario where a user recently searched for products, but perhaps didn't commit to a purchase yet. We now want to send a message to promote relevant products.

In [ ]:
# Import relevant libraries
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

#Instantiate LLM
llm = ChatOpenAI(temperature=0, model_name=LLM, streaming=True)

### Create Knowledge Graph Stores for Retrieval

To ground our content generation, we need to define retrievers to pull information from our knowledge graph.  Let's make two stores:
1. Personalized Search Retriever (`kg_personalized_search`): Based on recent customer searches and purchase history, pull relevant products.
2. Recommendations retriever (`kg_recommendations_app`): Based on our Graph ML, what else can we recommend to them to pair with the relevant products?


In [ ]:
# This will be a function so we can change per customer id
# We will use a mock URL for our sources in the metadata
def kg_personalized_search_gen(customer_id):
    return Neo4jVector.from_existing_index(
        embedding=embedding_model,
        url=NEO4J_URI,
        username=NEO4J_USERNAME,
        password=NEO4J_PASSWORD,
        index_name='product_text_embeddings',
        retrieval_query=f"""
        WITH node AS product, score AS searchScore

        OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
        -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {{customerId: '{customer_id}'}})
        WITH count(a) AS purchaseScore, product, searchScore
        RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
            (1.0+purchaseScore)*searchScore AS score,
            {{source: 'https://representative-domain/product/' + product.productCode}} AS metadata
        ORDER BY purchaseScore DESC, searchScore DESC LIMIT 5

    """
    )

# Use the same personalized recommendations as above but with a smaller limit
def kg_recommendations_app(customer_id, k=30):
    res = kg.query("""
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.text + '\nurl: ' + 'https://representative-domain/product/' + product.productCode  AS text,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    """, params={'customerId': customer_id, 'k':k})

    return "\n\n".join([d['text'] for d in res])

### Prompt Engineering

Now, let's define our prompts. We will combine two:
1. A system prompt which, in this case, tells the LLM how to generate the message
2. A human prompt that just wraps the customer search(es)/interest(s)

This will allow us to pass the customer interest(s) to the retriever but then also to the LLM for additional context when drafting the message.


In [ ]:
general_system_template = '''
You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.
write an email to {customerName}, one of your customers, to promote and summarize products relevant for them given the current season / time of year: {timeOfYear} .
Please only mention the products listed below. Do not come up with or add any new products to the list.
Each product comes with an https `url` field. Make sure to provide that https url with descriptive name text in markdown for each product.

---
# Relevant Products:
{searchProds}

# Customer May Also Be Interested In the following
 (pick items from here that pair with the above products well for the current season / time of year: {timeOfYear}.
 prioritize those higher in the list if possible):
{recProds}
---
'''
general_user_template = "{searchPrompt}"
messages = [
    SystemMessagePromptTemplate.from_template(general_system_template),
    HumanMessagePromptTemplate.from_template(general_user_template),
]
prompt = ChatPromptTemplate.from_messages(messages)

### Create a Chain

Now let's put a chain together that will leverage the retrievers, prompts, and LLM model. This is where Langchain shines, putting RAG together in a simple way.

In addition to the personalized search and recommendations context, we will allow for some other parameters.

1. `timeOfYear`: The time of year as a date, season, month, etc. so the LLM can tailor the language appropriately.
2. `customerName`: Ordinarily, this can be pulled from the DB, but it has been scrubbed to maintain anonymity, so we will provide our own name here.

You can potentially add other creative parameters here to help the LLM write relevant messages.


In [ ]:
from langchain.schema.runnable import RunnableLambda

# helper function
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# LLM chain
def chain_gen(customer_id):
    return ({'searchProds': (lambda x:x['searchPrompt']) | kg_personalized_search_gen(customer_id).as_retriever(search_kwargs={"k": 100}) | format_docs,
             'recProds': (lambda x:customer_id) |  RunnableLambda(kg_recommendations_app),
             'customerName': lambda x:x['customerName'],
             'timeOfYear': lambda x:x['timeOfYear'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt
            | llm
            | StrOutputParser())

### Example Runs

In [ ]:
chain = chain_gen(CUSTOMER_ID)

In [ ]:
print(chain.invoke({'searchPrompt':search_prompt, 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

#### Inspecting the Prompt Sent to the LLM
In the above run, the LLM should only be using results from our Neo4j database to populate recommendations. Run the below cell to see the final prompt that was sent to the LLM.

In [ ]:
def format_final_prompt(x):
   return f'''=== Prompt to send to LLM ===
   {x.to_string()}
   === End Prompt ===
   '''

def chain_print_prompt(customer_id):
    return ({'searchProds': (lambda x:x['searchPrompt']) | kg_personalized_search_gen(customer_id).as_retriever(search_kwargs={"k": 100}) | format_docs,
             'recProds': (lambda x:customer_id) |  RunnableLambda(kg_recommendations_app),
             'customerName': lambda x:x['customerName'],
             'timeOfYear': lambda x:x['timeOfYear'],
             "searchPrompt":  lambda x:x['searchPrompt']}
            | prompt
            | format_final_prompt
            | StrOutputParser())

print( chain_print_prompt(CUSTOMER_ID)\
      .invoke({'searchPrompt':search_prompt, 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

Feel free to experiment and try more!

In [ ]:
print(chain.invoke({'searchPrompt':"western boots", 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

### Demo App
Now let’s use the above tools to create a demo app with Gradio.  We will need to make a couple more functions, but otherwise easy to fire up from a Notebook!

In [ ]:
# Create a means to generate and cache chains...so we can quickly try different customer ids
personalized_search_chain_cache = dict()
def get_chain(customer_id):
    if customer_id in personalized_search_chain_cache:
        return personalized_search_chain_cache[customer_id]
    chain = chain_gen(customer_id)
    personalized_search_chain_cache[customer_id] = chain
    return chain

# create multiple demo examples to try
examples = [
    [
        CUSTOMER_ID,
        'Feb, 2024',
        'Alex Smith',
        'Oversized Sweaters'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'Feb, 2024',
        'Robin Fischer',
        'Oversized Sweaters'
    ],
    [
        '44b0898ecce6cc1268dfdb0f91e053db014b973f67e34ed8ae28211410910693',
        'Feb, 2024',
        'Chris Johnson',
        'Oversized Sweaters'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'Feb, 2024',
        'Robin Fischer',
        'denim jeans'
    ],
]

In [ ]:
import gradio as gr

def message_generator(*x):
    chain = get_chain(x[0])
    return chain.invoke({'searchPrompt':x[3], 'customerName':x[2], 'timeOfYear': x[1]})

customer_id = gr.Textbox(value=CUSTOMER_ID, label="Customer ID")
time_of_year = gr.Textbox(value="Feb, 2024", label="Time Of Year")
search_prompt_txt = gr.Textbox(value='Oversized Sweaters', label="Customer Interests(s)")
customer_name = gr.Textbox(value='Alex Smith', label="Customer Name")
message_result = gr.Markdown( label="Message")

demo = gr.Interface(fn=message_generator,
                    inputs=[customer_id, time_of_year, customer_name, search_prompt_txt],
                    outputs=message_result,
                    examples=examples,
                    title="🪄 Message Generator 🥳")
demo.launch(share=True, debug=True)

## That's a Wrap!